In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

sys.path.append('/coc/scratch/anarayanan68/mint/')

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, initializers, regularizers

from mint.core import fact_model, base_model_util, base_models, primitive_models, model_builder
from mint.utils import inputs_util, config_util

In [4]:
config_path = '/coc/scratch/anarayanan68/mint/configs/audio_based_blending__embed_based_conditioning.config'
enc_config_path = '/coc/scratch/anarayanan68/mint/configs/audio_based_blending__embed_based_conditioning-enc_config.yml'

In [5]:
expt_root = '/coc/scratch/anarayanan68/mint/_expts/audio_based_blending__embed_based_conditioning'

checkpoint_dir = os.path.join(expt_root, 'checkpoints/')

In [6]:
configs = config_util.get_configs_from_pipeline_file(config_path)
model_config = configs['model']
eval_config = configs['eval_config']
eval_dataset_config = configs['eval_dataset']

In [7]:
enc_config_yaml = config_util.read_yaml_config(enc_config_path)

In [8]:
# Model build & restore

model = model_builder.build(model_config, is_training=False,
    encoder_config_yaml=enc_config_yaml, dataset_config=eval_dataset_config)

checkpoint_manager=tf.train.CheckpointManager(
    tf.train.Checkpoint(model=model),
    directory=checkpoint_dir,
    max_to_keep=None)

checkpoint_manager.restore_or_initialize()

'/coc/scratch/anarayanan68/mint/_expts/audio_based_blending__embed_based_conditioning/checkpoints/ckpt-9999'

In [9]:
num_primitives = enc_config_yaml['num_primitives']
all_primitives = model.blend_vec_to_seq(np.eye(num_primitives))
all_primitives

<tf.Tensor: shape=(100, 120, 147), dtype=float32, numpy=
array([[[ 0.00752957,  0.00752949,  0.00753065, ...,  0.00753002,
          0.00752935,  0.00753043],
        [ 0.00752955,  0.00752941,  0.00753067, ...,  0.00752984,
          0.00752928,  0.00753035],
        [ 0.00752959,  0.00752945,  0.00753066, ...,  0.00752997,
          0.00752928,  0.00753038],
        ...,
        [ 0.00752949,  0.00752952,  0.00753072, ...,  0.0075298 ,
          0.00752941,  0.00753032],
        [ 0.00752958,  0.00752934,  0.00753075, ...,  0.00752984,
          0.0075293 ,  0.00753036],
        [ 0.00752943,  0.00752938,  0.00753063, ...,  0.00752999,
          0.00752925,  0.00753031]],

       [[ 0.00753027,  0.0075299 ,  0.00753392, ...,  0.00753153,
          0.00752969,  0.00753291],
        [ 0.00753021,  0.00752969,  0.0075339 , ...,  0.00753104,
          0.00752953,  0.00753271],
        [ 0.00753034,  0.00752988,  0.00753395, ...,  0.00753145,
          0.00752953,  0.00753282],
        ..

In [10]:
save_dir = os.path.join(expt_root, 'primitives')
os.makedirs(save_dir, exist_ok=True)
for i in range(num_primitives):
    np.save(os.path.join(save_dir, f"PRIMITIVE--{i+1}_of_{num_primitives}.npy"), all_primitives[i])